In [8]:
%ls

Makefile                          matrices/
README.md                         mlir_files/
Report.md                         python/
Untitled.ipynb                    results_scalar/
__pycache__/                      results_vector/
all_results_scalar.csv            scalar_pipeline.sh*
all_results_vector/               src/
analysis_output/                  stride_comparison.png
analysis_output_stride1/          stride_grouped_analysis/
folder_structure.txt              vector_performance_dashboard.png
jupiter/                          vector_pipeline.sh*


In [9]:
import os
import re
import pandas as pd

# Directory principale contenente le sottocartelle
directory = "results_vector"  # Cartella principale

# Lista per contenere i dataframe da tutti i file
all_dataframes = []

# Leggi tutti i file CSV dalle sottocartelle
for subdir in os.listdir(directory):
    subdir_path = os.path.join(directory, subdir)
    
    # Verifica che sia una directory e non un file
    if os.path.isdir(subdir_path):
        print(f"Analisi della sottocartella: {subdir}")
        
        # Lista per contenere i dataframe della sottocartella corrente
        subdir_dataframes = []
        
        # Estrai la dimensione del vettore dal nome della sottocartella
        vector_size_match = re.search(r"results_vector_(\d+)", subdir)
        vector_size = int(vector_size_match.group(1)) if vector_size_match else None
        
        # Leggi tutti i file CSV nella sottocartella
        for filename in os.listdir(subdir_path):
            if filename.endswith(".csv"):
                filepath = os.path.join(subdir_path, filename)
                print(f"  Elaborazione file: {filename}")
                
                try:
                    df = pd.read_csv(filepath)
                    
                    # Standardizza le colonne
                    df.columns = ['Value', 'Unit', 'Metric', 'Percentage', 'Count', 'Total', 'Rate', 'Description']
                    df.replace('', pd.NA, inplace=True)
                    df.dropna(subset=['Value'], inplace=True)
                    df['Value'] = pd.to_numeric(df['Value'])
                    df['Rate'] = pd.to_numeric(df['Rate'])
                    
                    # Aggiungi indice fittizio per il pivot
                    df['DummyIndex'] = range(len(df))
                    
                    # Pivot del DataFrame
                    df_pivot = df.pivot(index='DummyIndex', columns='Metric', values='Value')
                    df_pivot = df_pivot.reset_index(drop=True)
                    df_pivot.columns.name = None
                    
                    # Rinomina le metriche per maggiore chiarezza
                    translation_dict = {
                        'task-clock': 'Task Clock',
                        'context-switches': 'Context Switch',
                        'cpu-migrations': 'CPU Migration',
                        'page-faults': 'Page Faults',
                        'cycles': 'Cycles',
                        'instructions': 'Instructions',
                        'branches': 'Branches',
                        'branch-misses': 'Branch Misses'
                    }
                    df_pivot.rename(columns=translation_dict, inplace=True)
                    
                    # Estrai sparsity e stride dal nome del file
                    match = re.search(r"sparsity_(\d+)_stride_(\d+)", filename)
                    if match:
                        sparsity = int(match.group(1))
                        stride = int(match.group(2))
                    else:
                        sparsity = None
                        stride = None
                    
                    # Aggiungi i dati estratti al DataFrame
                    df_pivot['Executable'] = os.path.splitext(filename)[0]
                    df_pivot['Sparsity Level'] = sparsity
                    df_pivot['Stride'] = stride
                    df_pivot['Vector Size'] = vector_size  # Aggiungi la dimensione del vettore
                    
                    subdir_dataframes.append(df_pivot)
                    all_dataframes.append(df_pivot)
                except Exception as e:
                    print(f"  Errore nell'elaborazione del file {filename}: {e}")
        
        # Crea un file CSV per la sottocartella corrente
        if subdir_dataframes:
            # Unisci tutti i DataFrame della sottocartella
            subdir_df = pd.concat(subdir_dataframes, ignore_index=True)
            
            # Ordina il DataFrame per executable
            subdir_df_sorted = subdir_df.sort_values(by=['Executable'])
            
            # Reimposta l'indice
            subdir_df_sorted = subdir_df_sorted.reset_index(drop=True)
            
            # Salva il DataFrame in un file CSV specifico per la sottocartella
            subdir_csv_name = f"all_results_vector/all_{subdir}.csv"
            subdir_df_sorted.to_csv(subdir_csv_name)
            
            print(f"File CSV creato per la sottocartella {subdir}: {subdir_csv_name}")

# Crea un file CSV complessivo con tutti i dati
if all_dataframes:
    # Unisci tutti i DataFrame in uno
    final_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Ordina il DataFrame per dimensione del vettore e poi per nome dell'eseguibile
    final_df_sorted = final_df.sort_values(by=['Vector Size', 'Executable'])
    
    # Reimposta l'indice
    final_df_sorted = final_df_sorted.reset_index(drop=True)
    
    # Salva il DataFrame in un file CSV complessivo
    all_csv_name = f"all_results_vector/all_{directory}.csv"
    final_df_sorted.to_csv(all_csv_name)
    
    print(f"File CSV complessivo creato: {all_csv_name}")
else:
    print("Nessun dato trovato nelle cartelle specificate.")

Analisi della sottocartella: results_vector_32
  Elaborazione file: results_vector_mlir_sparsity_55_stride_1.csv
  Elaborazione file: results_vector_mlir_sparsity_55_stride_3.csv
  Elaborazione file: results_vector_mlir_sparsity_55_stride_2.csv
  Elaborazione file: results_vector_mlir_sparsity_70_stride_2.csv
  Elaborazione file: results_vector_mlir_sparsity_65_stride_3.csv
  Elaborazione file: results_vector_mlir_sparsity_65_stride_2.csv
  Elaborazione file: results_vector_mlir_sparsity_70_stride_3.csv
  Elaborazione file: results_vector_mlir_sparsity_70_stride_1.csv
  Elaborazione file: results_vector_mlir_sparsity_65_stride_1.csv
  Elaborazione file: results_vector_mlir_sparsity_90_stride_1.csv
  Elaborazione file: results_vector_mlir_sparsity_85_stride_1.csv
  Elaborazione file: results_vector_mlir_sparsity_90_stride_2.csv
  Elaborazione file: results_vector_mlir_sparsity_85_stride_3.csv
  Elaborazione file: results_vector_mlir_sparsity_85_stride_2.csv
  Elaborazione file: results_